In [1]:
API_KEY = "980a590f6e6e045b0aead2bb1eec07e5"

In [16]:
#!/usr/bin/env python

from urllib.request import urlopen

import certifi
import json

def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

url = (f"https://financialmodelingprep.com/api/v4/institutional-ownership/institutional-holders/symbol-ownership-percent?date=2021-09-30&symbol=AAPL&page=0&apikey={API_KEY}")
data = get_jsonparsed_data(url)

/var/folders/vm/gl7wrnv94t11c0w_b6_qby0w0000gn/T/ipykernel_28388/3320227258.py:10: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


In [31]:
class Company:
    def __init__(self, cik, name):
        self.__cik = cik
        self.__name = name
        self.__investors = []
    
    
    def add_investors(self, investor, percent):
        if isinstance(investor, Company) & isinstance(percent, float):
            self.__investors.append((investor, percent))
        else:
            raise Exception('invalid type of investor and percent') 
    
    
    def get_investors(self):
        return self.__investors

    
    def get_name(self):
        return self.__name
    
    
    def get_cik(self):
        return self.__cik
    
    

In [32]:
def is_company_in_by_cik(list, cik):
    for c in list:
        if c.get_cik() == cik:
            return c
    return False

In [33]:
r = {}

companies = []
apple = Company(int("0000320193"), "Apple Inc.")
companies.append(apple)

for i in data:
    cik = int(i["cik"])
    c = is_company_in_by_cik(companies, cik)

    if c is False:
        cname = i["investorName"]
        c = Company(cik, cname)
    
    p = float(i["ownership"])
    
    apple.add_investors(c, p)

In [34]:
apple.get_investors()

[(<__main__.Company at 0x10fadfd50>, 7.5087),
 (<__main__.Company at 0x10fc34d90>, 6.085),
 (<__main__.Company at 0x10fc34ad0>, 5.2602),
 (<__main__.Company at 0x10f8ac410>, 3.6891),
 (<__main__.Company at 0x10f8aebd0>, 2.079),
 (<__main__.Company at 0x10f8afd50>, 1.541),
 (<__main__.Company at 0x10f8acd90>, 1.1582),
 (<__main__.Company at 0x10f8ae310>, 1.1176),
 (<__main__.Company at 0x10f8ae490>, 0.8858),
 (<__main__.Company at 0x10f8af3d0>, 0.8666),
 (<__main__.Company at 0x10f8ac150>, 0.8042),
 (<__main__.Company at 0x10f8ae5d0>, 0.7606),
 (<__main__.Company at 0x10f8afb50>, 0.7358),
 (<__main__.Company at 0x10f8ac590>, 0.6381),
 (<__main__.Company at 0x10f8aecd0>, 0.5251),
 (<__main__.Company at 0x10f8ac310>, 0.4694),
 (<__main__.Company at 0x10f8af6d0>, 0.4643),
 (<__main__.Company at 0x10f8afb10>, 0.4139),
 (<__main__.Company at 0x10f8af510>, 0.4105),
 (<__main__.Company at 0x10f8af0d0>, 0.3913),
 (<__main__.Company at 0x10f8ad990>, 0.3795),
 (<__main__.Company at 0x10f8aef10>, 

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()

for company in companies:
    for c, p in company.get_investors():
        G.add_edge(company.get_cik(), c.get_cik(), weight=p)

pos = nx.spring_layout(G, seed=42)

size = [degree * 15 for node, degree in G.degree]
color = [degree* 5 for node, degree in G.degree]

fig, ax = plt.subplots(2, 2)

nx.draw_networkx(G, pos=pos, ax=ax, with_labels=False, node_size=size, node_color=color, cmap=plt.cm.Greens, edge_color='grey')
    